# EXC RB simulation

Tim Keil

In [1]:
 # ~~~
 # This file is part of the paper:
 #
 #   "Model Reduction for Large Scale Systems"
 #
 #   https://github.com/TiKeil/Petrov-Galerkin-TR-RB-for-pde-opt
 #
 # Copyright 2019-2021 all developers. All rights reserved.
 # License: Licensed as BSD 2-Clause License (http://opensource.org/licenses/BSD-2-Clause)
 # Authors:
 #   Tim Keil (2019 - 2021)
 # ~~~

# Preparations

## details

In [2]:
import sys
path = '../../../EXC_data'
sys.path.append(path)
import numpy as np

from matplotlib import pyplot as plt

from pymor.basic import *
set_log_levels({'pymor': 'WARN'})
set_defaults({'pymor.operators.constructions.induced_norm.raise_negative': False})
set_defaults({'pymor.operators.constructions.induced_norm.tol': 1e-20})

In [ ]:
from pymor.core.logger import set_log_levels, getLogger
set_log_levels({'pymor': 'ERROR',
                'distributed_adaptive_discretizations': 'DEBUG',
                'notebook': 'INFO'})
logger = getLogger('notebook.notebook')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12.0, 8.0)
mpl.rcParams['font.size'] = 12
mpl.rcParams['savefig.dpi'] = 300

# domain of interest
bounding_box = [[0,0],[2,1]]
domain_of_interest = ConstantFunction(1,2)     # <-- THIS IS THE DUAL SIMPLIFICATION domain_of_interest = Omega
# domain_of_interest = BitmapFunction(f'{path}/Domain_of_interest.png', range=[1,0], bounding_box=bounding_box)

## problem definition

In [4]:
from pdeopt.problems import EXC_problem, set_input_dict
from pdeopt.discretizer import discretize_quadratic_pdeopt_stationary_cg

parametric_quantities = {'walls': [1,4,9], 'windows': [], 'doors': [6,7], 'heaters': [1,3,5,6,7,8,9]}
inactive_quantities = {'removed_walls': [], 'open_windows': [], 'open_doors': [1,2,3,4,5,10], 'active_heaters': []}
summed_quantities = {'walls': [[1,2,3,7,8],[4,5,6]], 'windows': [], 'doors': [], 'heaters': [[1,2],[3,4],[9,10,11,12]]}

coefficient_expressions = None

parameters_in_q = True
input_dict = set_input_dict(parametric_quantities, inactive_quantities, coefficient_expressions, summed_quantities, parameters_in_q,
                            ac=0.5, owc=[0.025,0.1], iwc= [0.025,0.1], idc=[0.05,0.2], wc=[0.0005], ht=[0,100],
                                    owc_c=0.001,  iwc_c= 0.025,     idc_c=0.01,  wc_c=0.05,  ht_c=80)


parameter_scaling = False
u_out = 5

problem, parameter_scales = EXC_problem(input_dict, summed_quantities, outside_temperature=u_out, #, q_inverse=0.0001
                                        data_path = path,parameters_in_q=parameters_in_q, 
                                        parameter_scaling=parameter_scaling,
                                        coefficient_expressions=coefficient_expressions)

u_d = 18 

mu_d = problem.parameter_space.sample_randomly(1, seed=10)[0]
mu_d = None

sigma_d = 100
weights = {'walls': 0.1, 'doors': 1, 'heaters': [0.002,0.002,0.0005,0.0005,0.0005,0.0005,0.004], 'windows': 1, 'state': sigma_d}

diameter = np.sqrt(2)/200.
opt_fom, data, mu_bar = discretize_quadratic_pdeopt_stationary_cg(problem, diameter, weights, parameter_scales, 
                                                          domain_of_interest, desired_temperature=u_d, 
                                                          mu_for_u_d=mu_d, mu_for_tikhonov=mu_d,
                                                          parameters_in_q=parameters_in_q, product='fixed_energy')

I am using the NCD corrected functional!!
my product is fixed_energy
mu_bar is: {doors: [0.1, 0.1], heaters: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0], walls: [0.049999999999999996, 0.049999999999999996, 0.049999999999999996]}


In [5]:
print('information on the grid:')
print(data['grid'])

N = [4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60]
validation_set_size = 100
tau_global_RB_J = 1e-12

information on the grid:
Rect-Grid on domain [0,2] x [0,1]
x0-intervals: 400, x1-intervals: 200
faces: 80000, edges: 160600, vertices: 80601


# Classical Model Order Reduction

### BFGS-Greedy for non-corrected functional

We now construct a simple RB basis for primal, dual and all sensitivities. For this, we start with an empty basis

In [6]:
params = []
from pdeopt.model import build_initial_basis
RBbasis, dual_RBbasis = build_initial_basis(opt_fom, params, build_sensitivities=False)

from pdeopt.reductor import QuadraticPdeoptStationaryCoerciveReductor
from pdeopt.pg_reductor import QuadraticPdeoptStationaryCoercivePGReductor

from pymor.parameters.functionals import MinThetaParameterFunctional

ce = MinThetaParameterFunctional(opt_fom.primal_model.operator.coefficients, mu_bar)

# opt_fom = opt_fom.with_(use_corrected_functional=False)
# opt_fom = opt_fom.with_(adjoint_approach=False)

# pdeopt_reductor = QuadraticPdeoptStationaryCoerciveReductor(opt_fom, 
#                                 RBbasis, dual_RBbasis, opt_product=opt_fom.opt_product,
#                                 coercivity_estimator=ce, mu_bar=mu_bar)

opt_fom = opt_fom.with_(use_corrected_functional=False)
opt_fom = opt_fom.with_(adjoint_approach=False)
opt_fom = opt_fom.with_(petrov_galerkin=True) 

pg_opt_reductor = QuadraticPdeoptStationaryCoercivePGReductor(opt_fom, 
                        RBbasis, dual_RBbasis, opt_product=opt_fom.opt_product,
                        coercivity_estimator=ce, mu_bar=mu_bar, least_squares=False)

Starting with two bases. Primal and dual have length 0 and 0
building simple coercive primal reductor...
building simple coercive dual reductor...


We start a greedy for the whole domain

In [7]:
set_log_levels({'pymor': 'WARN'})  # <-- set this to 'INFO' if you want to have further details 
from pdeopt.greedy import pdeopt_adaptive_greedy_for_N_list
training_set = problem.parameter_space.sample_randomly(10000)

opt_roms_3, result_J = pdeopt_adaptive_greedy_for_N_list(
    opt_fom, pg_opt_reductor, problem.parameter_space, Nlist=N, validation_mus=training_set, 
    J_atol=tau_global_RB_J)
tictoc = result_J['time']
print('Greedy took {}'.format(tictoc))
picked_mus = result_J['max_err_mus']
assert len(picked_mus) == N[-1]

Global Greedy for J target

Processing until 4th extension...
building simple coercive dual reductor...
Enrichment completed... length of Bases are 1 and 1
building simple coercive dual reductor...
Enrichment completed... length of Bases are 2 and 2
building simple coercive dual reductor...
Enrichment completed... length of Bases are 3 and 3
building simple coercive dual reductor...
Enrichment completed... length of Bases are 4 and 4

Processing until 8th extension...
building simple coercive dual reductor...
Enrichment completed... length of Bases are 5 and 5
building simple coercive dual reductor...
Enrichment completed... length of Bases are 6 and 6
building simple coercive dual reductor...
Enrichment completed... length of Bases are 7 and 7
building simple coercive dual reductor...
Enrichment completed... length of Bases are 8 and 8

Processing until 12th extension...
building simple coercive dual reductor...
Enrichment completed... length of Bases are 9 and 9
building simple coerc

In [8]:
print(f'The max error was: ', result_J['max_errs'][-1])

The max error was:  0.0027352573585843016


In [9]:
params = picked_mus
RBbasis, dual_RBbasis = build_initial_basis(opt_fom, params, build_sensitivities=False)

### Uncorrected Model

In [10]:
opt_fom = opt_fom.with_(use_corrected_functional=False)
opt_fom = opt_fom.with_(adjoint_approach=False)
opt_fom = opt_fom.with_(petrov_galerkin=False) 

pdeopt_reductor = QuadraticPdeoptStationaryCoerciveReductor(opt_fom, 
                                RBbasis, dual_RBbasis, opt_product=opt_fom.opt_product,
                                coercivity_estimator=ce, mu_bar=mu_bar)

opt_roms_1 = []
for N_ in N:
    opt_roms_1.append(pdeopt_reductor.reduce_to_subbasis(N_))

Starting with two bases. Primal and dual have length 60 and 60
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 4 and 4
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 8 and 8
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 12 and 12
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 16 and 16
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 20 and 20
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 24 and 24
building simple coercive primal reductor...
bui

### NCD-corrected

In [11]:
from pymor.parameters.functionals import MinThetaParameterFunctional

ce = MinThetaParameterFunctional(opt_fom.primal_model.operator.coefficients, mu_bar)

opt_fom = opt_fom.with_(use_corrected_functional=True)
opt_fom = opt_fom.with_(adjoint_approach=True)

ncd_opt_reductor = QuadraticPdeoptStationaryCoerciveReductor(opt_fom, 
                            RBbasis, dual_RBbasis, opt_product=opt_fom.opt_product,
                            coercivity_estimator=ce, mu_bar=mu_bar)


opt_roms_2 = []
for N_ in N:
    opt_roms_2.append(ncd_opt_reductor.reduce_to_subbasis(N_))

Starting with two bases. Primal and dual have length 60 and 60
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 4 and 4
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 8 and 8
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 12 and 12
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 16 and 16
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 20 and 20
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 24 and 24
building simple coercive primal reductor...
bui

### Model LSQ PG

In [12]:
opt_fom = opt_fom.with_(use_corrected_functional=False)
opt_fom = opt_fom.with_(adjoint_approach=False)
opt_fom = opt_fom.with_(petrov_galerkin=True) 

lsq_opt_reductor = QuadraticPdeoptStationaryCoercivePGReductor(opt_fom, 
                        RBbasis, dual_RBbasis, opt_product=opt_fom.opt_product,
                        coercivity_estimator=ce, mu_bar=mu_bar, least_squares=True)

opt_roms_4 = []
for N_ in N:
    opt_roms_4.append(lsq_opt_reductor.reduce_to_subbasis(N_))

Starting with two bases. Primal and dual have length 60 and 60
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 4 and 4
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 8 and 8
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 12 and 12
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 16 and 16
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 20 and 20
building simple coercive primal reductor...
building simple coercive dual reductor...
Starting with two bases. Primal and dual have length 24 and 24
building simple coercive primal reductor...
bui

## compute true errors and estimators

In [13]:
validation_set = problem.parameter_space.sample_randomly(validation_set_size, seed=9)

In [14]:
from pdeopt.tools import compute_all_errors_and_estimators_for_all_ROMS

J_errors_1s, rel_J_errors_1s, J_estimators_1s, effectivities_J_1s, \
J_errors_2s, rel_J_errors_2s, J_estimators_2s, effectivities_J_2s, \
J_errors_3s, rel_J_errors_3s, J_estimators_3s, effectivities_J_3s, \
J_errors_4s, rel_J_errors_4s, J_estimators_4s, effectivities_J_4s, \
DJ_errors_1s, rel_DJ_errors_1s, \
DJ_errors_2s, rel_DJ_errors_2s, \
DJ_errors_3s, rel_DJ_errors_3s, \
DJ_errors_4s, rel_DJ_errors_4s, \
Js, \
u_errors_4s, rel_u_errors_4s, u_estimators_4s, effectivities_u_4s, \
u_errors_2s, rel_u_errors_2s, u_estimators_2s, effectivities_u_2s, \
p_errors_4s, rel_p_errors_4s, p_estimators_4s, effectivities_p_4s, \
p_errors_2s, rel_p_errors_2s, p_estimators_2s, effectivities_p_2s \
        = compute_all_errors_and_estimators_for_all_ROMS(
    validation_set, opt_fom, opt_roms_1, opt_roms_2, opt_roms_3, opt_roms_4,
    pg_opt_reductor, ncd_opt_reductor)

computing expensive FOM parts
....................................................................................................
computing the rest
..

/home/t_keil02/lrb-for-pde-opt-code/pymor/src/pymor/operators/numpy.py:322: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  R, _, _, _ = np.linalg.lstsq(self.matrix, V.to_numpy().T)


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [15]:
J_data_for_N = []
DJ_data_for_N = []
u_data_for_N = []
p_data_for_N = []

for (
J_errors_1, rel_J_errors_1, J_estimators_1, effectivities_J_1,
J_errors_2, rel_J_errors_2, J_estimators_2, effectivities_J_2,
J_errors_3, rel_J_errors_3, J_estimators_3, effectivities_J_3,
J_errors_4, rel_J_errors_4, J_estimators_4, effectivities_J_4,
DJ_errors_1, rel_DJ_errors_1, \
DJ_errors_2, rel_DJ_errors_2, \
DJ_errors_3, rel_DJ_errors_3, \
DJ_errors_4, rel_DJ_errors_4, \
J,
u_errors_4, rel_u_errors_4, u_estimators_4, effectivities_u_4,
u_errors_2, rel_u_errors_2, u_estimators_2, effectivities_u_2,
p_errors_4, rel_p_errors_4, p_estimators_4, effectivities_p_4,
p_errors_2, rel_p_errors_2, p_estimators_2, effectivities_p_2 
) in zip(
J_errors_1s, rel_J_errors_1s, J_estimators_1s, effectivities_J_1s,
J_errors_2s, rel_J_errors_2s, J_estimators_2s, effectivities_J_2s,
J_errors_3s, rel_J_errors_3s, J_estimators_3s, effectivities_J_3s,
J_errors_4s, rel_J_errors_4s, J_estimators_4s, effectivities_J_4s,
DJ_errors_1s, rel_DJ_errors_1s, \
DJ_errors_2s, rel_DJ_errors_2s, \
DJ_errors_3s, rel_DJ_errors_3s, \
DJ_errors_4s, rel_DJ_errors_4s, \
Js, 
u_errors_4s, rel_u_errors_4s, u_estimators_4s, effectivities_u_4s,
u_errors_2s, rel_u_errors_2s, u_estimators_2s, effectivities_u_2s,
p_errors_4s, rel_p_errors_4s, p_estimators_4s, effectivities_p_4s,
p_errors_2s, rel_p_errors_2s, p_estimators_2s, effectivities_p_2s 
):
    #J
    max_J_err_1 = max(J_errors_1)
    min_J_err_1 = min(J_errors_1)

    max_J_err_2 = max(J_errors_2)
    min_J_err_2 = min(J_errors_2)

    max_J_err_3 = max(J_errors_3)
    min_J_err_3 = min(J_errors_3)

    max_J_err_4 = max(J_errors_4)
    min_J_err_4 = min(J_errors_4)
        
    #J estimator
    max_J_est_1 = max(J_estimators_1)
    min_J_est_1 = min(J_estimators_1)

    max_J_est_2 = max(J_estimators_2)
    min_J_est_2 = min(J_estimators_2)

    max_J_est_3 = max(J_estimators_3)
    min_J_est_3 = min(J_estimators_3)

    max_J_est_4 = max(J_estimators_4)
    min_J_est_4 = min(J_estimators_4)
    
    max_eff_J_1 = max(effectivities_J_1)
    max_eff_J_2 = max(effectivities_J_2)
    max_eff_J_3 = max(effectivities_J_3)
    max_eff_J_4 = max(effectivities_J_4)

    J_data_for_N.append([
        [max_J_err_1, max_J_est_1, min_J_est_1, min_J_err_1, max_eff_J_1],
        [max_J_err_2, max_J_est_2, min_J_est_2, min_J_err_2, max_eff_J_2],
        [max_J_err_3, max_J_est_3, min_J_est_3, min_J_err_3, max_eff_J_3],
        [max_J_err_4, max_J_est_4, min_J_est_4, min_J_err_4, max_eff_J_4]])
    
    #DJ
    max_DJ_err_1 = max(DJ_errors_1)
    min_DJ_err_1 = min(DJ_errors_1)

    max_DJ_err_2 = max(DJ_errors_2)
    min_DJ_err_2 = min(DJ_errors_2)

    max_DJ_err_3 = max(DJ_errors_3)
    min_DJ_err_3 = min(DJ_errors_3)

    max_DJ_err_4 = max(DJ_errors_4)
    min_DJ_err_4 = min(DJ_errors_4)
    
    DJ_data_for_N.append([
        [max_DJ_err_1, min_DJ_err_1],
        [max_DJ_err_2, min_DJ_err_2],
        [max_DJ_err_3, min_DJ_err_3],
        [max_DJ_err_4, min_DJ_err_4]])
    
    # primal
    max_u_err_4 = max(u_errors_4)
    min_u_err_4 = min(u_errors_4)

    max_u_err_2 = max(u_errors_2)
    min_u_err_2 = min(u_errors_2)

    max_u_est_4 = max(u_estimators_4)
    min_u_est_4 = min(u_estimators_4)

    max_u_est_2 = max(u_estimators_2)
    min_u_est_2 = min(u_estimators_2)
    
    max_eff_u_4 = max(effectivities_u_4)
    max_eff_u_2 = max(effectivities_u_2)

    u_data_for_N.append([
        [max_u_err_2, max_u_est_2, min_u_est_2, min_u_err_2, max_eff_u_2],
        [max_u_err_4, max_u_est_4, min_u_est_4, min_u_err_4, max_eff_u_4]])
    
    # dual
    max_p_err_4 = max(p_errors_4)
    min_p_err_4 = min(p_errors_4)

    max_p_err_2 = max(p_errors_2)
    min_p_err_2 = min(p_errors_2)

    max_p_est_4 = max(p_estimators_4)
    min_p_est_4 = min(p_estimators_4)

    max_p_est_2 = max(p_estimators_2)
    min_p_est_2 = min(p_estimators_2)

    max_eff_p_4 = max(effectivities_p_4)
    max_eff_p_2 = max(effectivities_p_2)
    
    p_data_for_N.append([
        [max_p_err_2, max_p_est_2, min_p_est_2, min_p_err_2, max_eff_p_2],
        [max_p_err_4, max_p_est_4, min_p_est_4, min_p_err_4, max_eff_p_4]])


# Results

## J functional

In [16]:
from tabulate import tabulate

# tabulate for the output functional
print('max J error comparison')
print()
headers = ['basis size', '1 non corr', '2 ncd corr', '3 pg', '4 pg lsq']
table = []
for i, N_ in enumerate(N):
    data = J_data_for_N[i]
    table.append([N_, data[0][0], data[1][0], data[2][0], data[3][0]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

max J error comparison

|   basis size |   1 non corr |   2 ncd corr |      3 pg |   4 pg lsq |
|--------------|--------------|--------------|-----------|------------|
|            4 |   43.0423546 |    4.3567802 | 3.9085827 |  3.9085827 |
|            8 |    4.7622899 |    0.9740319 | 1.2450593 |  1.2450593 |
|           12 |    3.8108226 |    0.7101863 | 0.7329396 |  0.7329396 |
|           16 |    2.3004248 |    0.2501180 | 0.3251995 |  0.3251995 |
|           20 |    0.9208100 |    0.0288619 | 0.0247842 |  0.0247842 |
|           24 |    0.1874082 |    0.0110932 | 0.0110414 |  0.0110414 |
|           28 |    0.1138518 |    0.0053377 | 0.0072402 |  0.0072402 |
|           32 |    0.0583603 |    0.0025348 | 0.0094383 |  0.0094383 |
|           36 |    0.0300142 |    0.0010780 | 0.0029239 |  0.0029239 |
|           40 |    0.0269577 |    0.0002019 | 0.0019345 |  0.0019344 |
|           44 |    0.0338155 |    0.0001357 | 0.0011918 |  0.0011918 |
|           48 |    0.0187490 |    0.000

In [17]:
# tabulate for the output functional
print('max J estimator comparison')
print()
headers = ['basis size', '1 non corr', '2 ncd corr', '3 pg', '4 pg lsq']
table = []
for i, N_ in enumerate(N):
    data = J_data_for_N[i]
    table.append([N_, data[0][1], data[1][1], data[2][1], data[3][1]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

max J estimator comparison

|   basis size |   1 non corr |   2 ncd corr |         3 pg |     4 pg lsq |
|--------------|--------------|--------------|--------------|--------------|
|            4 | 3165.1702510 | 3141.2143878 | 3277.0454946 | 3277.0454946 |
|            8 |  264.4047738 |  264.2879019 |  545.0150992 |  545.0150992 |
|           12 |   37.3756536 |   37.0016027 |   64.5876599 |   64.5876599 |
|           16 |   24.7483852 |   24.6490924 |  246.4429782 |  246.4429783 |
|           20 |    8.5186668 |    8.1608033 |  151.7949229 |  151.7949229 |
|           24 |    0.5755556 |    0.4040536 |   17.5848934 |   17.5848934 |
|           28 |    0.2129657 |    0.1497437 |    0.4789816 |    0.4789816 |
|           32 |    0.1523195 |    0.1107386 |    0.8466747 |    0.8466747 |
|           36 |    0.0635487 |    0.0419511 |    0.2404652 |    0.2404652 |
|           40 |    0.0420447 |    0.0155598 |    0.6808230 |    0.6808230 |
|           44 |    0.0363611 |    0.0058217 |  

In [18]:
# tabulate for the output functional
print('J eff comparison')
print()
headers = ['basis size', '2 ncd corr', '3 pg']
table = []
for i, N_ in enumerate(N):
    data = J_data_for_N[i]
    table.append([N_, data[0][4], data[1][4]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f'))

J eff comparison

|   basis size |   2 ncd corr |          3 pg |
|--------------|--------------|---------------|
|            4 | 2780.5649445 | 28347.7240646 |
|            8 | 3304.8898695 |  6833.3901719 |
|           12 | 6663.4911970 |  3913.9630331 |
|           16 | 5379.7668375 |  5732.3049660 |
|           20 |  681.5003855 |  9557.1913710 |
|           24 |  206.2444367 | 35940.4294551 |
|           28 |  162.8181683 |  8328.2859095 |
|           32 |  369.2394059 | 11471.2290458 |
|           36 |   56.2191137 |  5801.3743077 |
|           40 |  515.9271523 | 14414.6747569 |
|           44 |   13.6578515 | 15038.8735065 |
|           48 |  182.0675118 |  1753.0961699 |
|           52 |   11.6191282 |  3150.0345512 |
|           56 |   14.7804744 |  2664.8572529 |
|           60 |    4.8136181 | 11375.3152282 |


## DJ 

In [19]:
print('max J error comparison')
print()
headers = ['basis size', '1 non corr', '2 ncd corr', '3 pg', '4 pg lsq']
table = []
for i, N_ in enumerate(N):
    data = DJ_data_for_N[i]
    table.append([N_, data[0][0], data[1][0], data[2][0], data[3][0]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

max J error comparison

|   basis size |   1 non corr |   2 ncd corr |        3 pg |    4 pg lsq |
|--------------|--------------|--------------|-------------|-------------|
|            4 |  113.8903117 |  116.8902273 | 116.9413910 | 116.9413910 |
|            8 |   78.8674009 |   71.2875380 |  81.3719586 |  81.3719586 |
|           12 |   66.5765970 |   53.3466076 |  43.7234391 |  43.7234391 |
|           16 |   41.4825594 |   21.9862603 |  37.3951660 |  37.3951660 |
|           20 |    3.3217361 |    1.4742002 |   1.3646116 |   1.3646116 |
|           24 |    0.9909078 |    0.4527781 |   0.6264351 |   0.6264351 |
|           28 |    0.6019228 |    0.2880695 |   0.3770512 |   0.3770512 |
|           32 |    0.5208526 |    0.1328267 |   0.4323930 |   0.4323930 |
|           36 |    0.3999896 |    0.0677626 |   0.1604367 |   0.1604367 |
|           40 |    0.0916279 |    0.0183905 |   0.0830978 |   0.0830978 |
|           44 |    0.0960969 |    0.0082919 |   0.1297149 |   0.1297149 |
|

## primal

In [20]:
# tabulate for the output functional
print('u error comparison')
print()
headers = ['basis size', '2 ncd corr', '3 pg']
table = []
for i, N_ in enumerate(N):
    data = u_data_for_N[i]
    table.append([N_, data[0][0], data[1][0]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

u error comparison

|   basis size |   2 ncd corr |      3 pg |
|--------------|--------------|-----------|
|            4 |    0.6979035 | 0.7062068 |
|            8 |    0.1683773 | 0.3404906 |
|           12 |    0.0880920 | 0.1114368 |
|           16 |    0.0661280 | 0.1809833 |
|           20 |    0.0349068 | 0.1392032 |
|           24 |    0.0068316 | 0.0474685 |
|           28 |    0.0045575 | 0.0073499 |
|           32 |    0.0036580 | 0.0077065 |
|           36 |    0.0026852 | 0.0051511 |
|           40 |    0.0012673 | 0.0101835 |
|           44 |    0.0009064 | 0.0090959 |
|           48 |    0.0005655 | 0.0016053 |
|           52 |    0.0003778 | 0.0006615 |
|           56 |    0.0003298 | 0.0005745 |
|           60 |    0.0002402 | 0.0006952 |


In [21]:
# tabulate for the output functional
print('u estimator comparison')
print()
headers = ['basis size', '2 ncd corr', '3 pg']
table = []
for i, N_ in enumerate(N):
    data = u_data_for_N[i]
    table.append([N_, data[0][1], data[1][1]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

u estimator comparison

|   basis size |   2 ncd corr |      3 pg |
|--------------|--------------|-----------|
|            4 |    1.1267376 | 1.1509328 |
|            8 |    0.3245847 | 0.4675477 |
|           12 |    0.1207877 | 0.1603159 |
|           16 |    0.0983618 | 0.3130725 |
|           20 |    0.0573171 | 0.2477959 |
|           24 |    0.0120203 | 0.0837140 |
|           28 |    0.0071521 | 0.0133642 |
|           32 |    0.0061419 | 0.0160391 |
|           36 |    0.0040785 | 0.0080779 |
|           40 |    0.0023548 | 0.0161891 |
|           44 |    0.0014752 | 0.0156381 |
|           48 |    0.0009369 | 0.0028227 |
|           52 |    0.0007637 | 0.0012630 |
|           56 |    0.0007043 | 0.0009514 |
|           60 |    0.0004982 | 0.0011383 |


In [22]:
# tabulate for the output functional
print('u eff comparison')
print()
headers = ['basis size', '2 ncd corr', '3 pg']
table = []
for i, N_ in enumerate(N):
    data = u_data_for_N[i]
    table.append([N_, data[0][4], data[1][4]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

u eff comparison

|   basis size |   2 ncd corr |      3 pg |
|--------------|--------------|-----------|
|            4 |    2.3370941 | 2.1009168 |
|            8 |    2.7008045 | 2.1902928 |
|           12 |    2.8514701 | 2.5112024 |
|           16 |    2.6425123 | 2.4048731 |
|           20 |    2.4331320 | 2.1785246 |
|           24 |    2.5701722 | 2.2981771 |
|           28 |    2.4724653 | 2.6184555 |
|           32 |    2.4967537 | 2.5018695 |
|           36 |    2.3312874 | 2.4064716 |
|           40 |    2.3376798 | 2.4770070 |
|           44 |    2.3116398 | 2.4189214 |
|           48 |    2.3190044 | 2.4230626 |
|           52 |    2.3070644 | 2.3285447 |
|           56 |    2.2856160 | 2.3518146 |
|           60 |    2.2815045 | 2.3705175 |


## dual

In [23]:
# tabulate for the output functional
print('p error comparison')
print()
headers = ['basis size', '2 ncd corr', '3 pg']
table = []
for i, N_ in enumerate(N):
    data = p_data_for_N[i]
    table.append([N_, data[0][0], data[1][0]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

p error comparison

|   basis size |   2 ncd corr |       3 pg |
|--------------|--------------|------------|
|            4 |   40.7460763 | 40.8146894 |
|            8 |   38.8899377 | 39.3312537 |
|           12 |   33.2418317 | 35.1527006 |
|           16 |   23.2686215 | 35.3298773 |
|           20 |    9.4626144 | 10.6842491 |
|           24 |    4.1539872 | 10.9024490 |
|           28 |    3.4412535 |  8.1679185 |
|           32 |    2.8128896 | 12.9165319 |
|           36 |    2.3952862 | 11.0609593 |
|           40 |    1.2969305 |  8.8701537 |
|           44 |    1.1588881 | 43.5757322 |
|           48 |    0.9383411 |  5.3983823 |
|           52 |    0.7765199 |  7.8105103 |
|           56 |    0.6220427 |  5.2246835 |
|           60 |    0.4001464 | 10.7762531 |


In [24]:
# tabulate for the output functional
print('p estimator comparison')
print()
headers = ['basis size', '2 ncd corr', '3 pg']
table = []
for i, N_ in enumerate(N):
    data = p_data_for_N[i]
    table.append([N_, data[0][1], data[1][1]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

p estimator comparison

|   basis size |   2 ncd corr |         3 pg |
|--------------|--------------|--------------|
|            4 | 3314.4915026 | 3384.8566080 |
|            8 | 1044.2122510 | 1490.4772010 |
|           12 |  369.4921049 |  481.0013244 |
|           16 |  302.9134902 |  949.3420514 |
|           20 |  170.0176986 |  729.7500855 |
|           24 |   45.8931937 |  251.4578324 |
|           28 |   29.1145765 |   46.8772780 |
|           32 |   25.0995237 |   76.5564492 |
|           36 |   13.4125369 |   36.9286502 |
|           40 |    9.0344776 |   51.5039700 |
|           44 |    5.2009862 |  105.0721066 |
|           48 |    4.0151371 |   13.1081584 |
|           52 |    3.6330931 |   20.6958803 |
|           56 |    3.3285899 |   10.4409919 |
|           60 |    2.4240827 |   16.9395369 |


In [25]:
# tabulate for the output functional
print('p eff comparison')
print()
headers = ['basis size', '2 ncd corr', '3 pg']
table = []
for i, N_ in enumerate(N):
    data = p_data_for_N[i]
    table.append([N_, data[0][4], data[1][4]])

print(tabulate(table, headers=headers, tablefmt='github', floatfmt='.7f')) 

p eff comparison

|   basis size |   2 ncd corr |        3 pg |
|--------------|--------------|-------------|
|            4 |  218.0635773 | 317.1192860 |
|            8 |  240.9262370 | 264.8889118 |
|           12 |  132.5182053 | 165.5789741 |
|           16 |  166.0367722 | 296.1618433 |
|           20 |  161.4564837 | 229.7975367 |
|           24 |  101.4277215 | 156.2206450 |
|           28 |   83.5570571 |  86.5507879 |
|           32 |   86.3793393 |  42.7870552 |
|           36 |  141.1268178 |  61.5356621 |
|           40 |  129.3485014 |  84.8022788 |
|           44 |   84.9175004 | 185.5953308 |
|           48 |   83.5895829 |  59.5990114 |
|           52 |   84.7123282 |  32.4075792 |
|           56 |   84.2958068 |  43.2952008 |
|           60 |   73.8084284 |  15.7799905 |
